
## Multivariate Analysis for Spatial Data Analytics in Python 


### Reidar B Bratvold,  Professor, University of Stavanger 

Multivariate Analysis for Subsurface Data Analytics in Python 

Here's a simple workflow, demonstration of multivariate analysis for subsurface modeling workflows. This should help you get started with building subsurface models that integrate uncertainty in the sample statistics.  

#### Bivariate Analysis

Understand and quantify the relationship between two variables

* example: relationship between porosity and permeability
* how can we use this relationship?

What would be the impact if we ignore this relationship and simply modeled porosity and permeability independently?

* no relationship beyond constraints at data locations
* independent away from data
* nonphysical results, unrealistic uncertainty models

#### Bivariate Statistics

Pearson’s Product‐Moment Correlation Coefficient
* Provides a measure of the degree of linear relationship.
* We refer to it as the 'correlation coefficient'

Let's review the sample variance of variable $x$. Of course, I'm truncating our notation as $x$ is a set of samples a locations in our modeling space, $x(\bf{u_\alpha}), \, \forall \, \alpha = 0, 1, \dots, n - 1$.

\begin{equation}
\sigma^2_{x}  = \frac{\sum_{i=1}^{n} (x_i - \overline{x})^2}{(n-1)}
\end{equation}

We can expand the the squared term and replace on of them with $y$, another variable in addition to $x$.

\begin{equation}
C_{xy}  = \frac{\sum_{i=1}^{n} (x_i - \overline{x})(y_i - \overline{y})}{(n-1)}
\end{equation}

We now have a measure that represents the manner in which variables $x$ and $y$ co-vary or vary together.  We can standardized the covariance by the product of the standard deviations of $x$ and $y$ to calculate the correlation coefficent. 

\begin{equation}
\rho_{xy}  = \frac{\sum_{i=1}^{n} (x_i - \overline{x})(y_i - \overline{y})}{(n-1)\sigma_x \sigma_y}, \, -1.0 \le \rho_{xy} \le 1.0
\end{equation}

In summary we can state that the correlation coefficient is related to the covariance as:

\begin{equation}
\rho_{xy}  = \frac{C_{xy}}{\sigma_x \sigma_y}
\end{equation}

The Pearson's correlation coefficient is quite sensitive to outliers and depature from linear behavoir (in the bivariate sense).  We have an altenrative known as the Spearman's rank correlations coefficient.   

\begin{equation}
\rho_{R_x R_y}  = \frac{\sum_{i=1}^{n} (R_{x_i} - \overline{R_x})(R_{y_i} - \overline{R_y})}{(n-1)\sigma_{R_x} \sigma_{R_y}}, \, -1.0 \le \rho_{xy} \le 1.0
\end{equation}

The rank correlation applies the rank transform to the data prior to calculating the correlation coefficent.  To calculate the rank transform simply replace the data values with the rank $R_x = 1,\dots,n$, where $n$ is the maximum value and $1$ is the minimum value. 

\begin{equation}
x_\alpha, \, \forall \alpha = 1,\dots, n, \, | \, x_i \ge x_j \, \forall \, i \gt j 
\end{equation}

\begin{equation}
R_{x_i} = i
\end{equation}

The corelation coefficients provide useful metrics to quantify relationships between two variables at a time. We can also consider bivariate scatter plots and matrix scatter plots to visualize multivariate data. In general, current practical subsurface modeling is bivariate, two variables at a time.    

#### Multivariate Statistics

See lecture on Multivariate Statistics, including the concepts of joint, conditional and marginal probability.

#### Objective 

To provide hands-on experience with building subsurface modeling workflows. Python provides an excellent vehicle to accomplish this. 

The objective is to remove the hurdles of subsurface modeling workflow construction by providing building blocks and sufficient examples. This is not a coding class per se, but we need the ability to 'script' workflows working with numerical methods.    


Import some some standard packages.

In [ ]:
import numpy as np                        # ndarrys for gridded data
import pandas as pd                       # DataFrames for tabular data
import os                                 # set working directory, run executables
import matplotlib.pyplot as plt           # for plotting
from scipy import stats                   # summary statistics
import math                               # trig etc.
import scipy.signal as signal             # kernel for moving window calculation
import random
import seaborn as sns

#### Loading Tabular Data

Here's the command to load our comma delimited data file in to a Pandas' DataFrame object.  

In [ ]:
df = pd.read_csv('sample_data_MV_biased.csv') # load our data table (wrong name!)

Visualizing the DataFrame would be useful and we already learned about these methods in this demo (https://git.io/fNgRW). 

We can preview the DataFrame by printing a slice or by utilizing the 'head' DataFrame member function (with a nice and clean format, see below). With the slice we could look at any subset of the data table and with the head command, add parameter 'n=13' to see the first 13 rows of the dataset.  

In [ ]:
print(df.iloc[0:5,:])                   # display first 4 samples in the table as a preview
df.head(n=13)                           # we could also use this command for a table preview

#### Summary Statistics for Tabular Data

The table includes X and Y coordinates (meters), Facies 1 and 0 (1 is sandstone and 0 interbedded sand and mudstone), Porosity (fraction), and permeability as Perm (mDarcy). 

There are a lot of efficient methods to calculate summary statistics from tabular data in DataFrames. The describe command provides count, mean, minimum, maximum, and quartiles all in a nice data table. We use transpose just to flip the table so that features are on the rows and the statistics are on the columns.

In [ ]:
df.describe().transpose()

#### Visualizing Tabular Data with Location Maps 

It is natural to set the x and y coordinate and feature ranges manually. e.g. do you want your color bar to go from 0.05887 to 0.24230 exactly? Also, let's pick a color map for display. I heard that plasma is known to be friendly to the color blind as the color and intensity vary together (hope I got that right, it was an interesting Twitter conversation started by Matt Hall from Agile if I recall correctly). We will assume a study area of 0 to 1,000m in x and y and omit any data outside this area.

In [ ]:
xmin = 0.0; xmax = 1000.0               # range of x values
ymin = 0.0; ymax = 1000.0               # range of y values
pormin = 0.05; pormax = 0.25;           # range of porosity values
permmin = 0.01; permmax = 2000.0         # range of permeability values
AImin = 2000.0; AImax = 8000.0          # range of AI values
nx = 100; ny = 100; csize = 10.0
cmap = plt.cm.plasma                    # color map

Let's try out locmap. This is a reimplementation of GSLIB's locmap program that uses matplotlib. I hope you find it simpler than matplotlib, if you want to get more advanced and build custom plots lock at the source. If you improve it, send me the new code. 

Now we can populate the plotting parameters and visualize the porosity data.

In [ ]:

# Create figure and subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Define color maps
cmap = 'plasma'  # Try 'viridis', 'inferno', or 'coolwarm' for different styles
marker_size = 10  # Adjust marker size

# Plot 1: Facies Well Data
sc = axes[0, 0].scatter(df['X'], df['Y'], c=df['Facies'], cmap=cmap, edgecolors='black', alpha=0.8, s=marker_size)
axes[0, 0].set_title("Well Data - Facies", fontsize=14)
axes[0, 0].set_xlabel("X (m)", fontsize=12)
axes[0, 0].set_ylabel("Y (m)", fontsize=12)
axes[0, 0].set_aspect('equal', adjustable='box')  # Ensure square plot
cb = plt.colorbar(sc, ax=axes[0, 0])
cb.set_label("Facies (0=shale, 1=sand)")

# Plot 2: Porosity Well Data
sc = axes[0, 1].scatter(df['X'], df['Y'], c=df['Porosity'], cmap=cmap, edgecolors='black', alpha=0.8, s=marker_size)
axes[0, 1].set_title("Well Data - Porosity", fontsize=14)
axes[0, 1].set_xlabel("X (m)", fontsize=12)
axes[0, 1].set_ylabel("Y (m)", fontsize=12)
axes[0, 1].set_aspect('equal', adjustable='box')  # Square plot
cb = plt.colorbar(sc, ax=axes[0, 1])
cb.set_label("Porosity (fraction)")

# Plot 3: Permeability Well Data
sc = axes[1, 0].scatter(df['X'], df['Y'], c=df['Perm'], cmap=cmap, edgecolors='black', alpha=0.8, s=marker_size)
axes[1, 0].set_title("Well Data - Permeability", fontsize=14)
axes[1, 0].set_xlabel("X (m)", fontsize=12)
axes[1, 0].set_ylabel("Y (m)", fontsize=12)
axes[1, 0].set_aspect('equal', adjustable='box')  # Square plot
cb = plt.colorbar(sc, ax=axes[1, 0])
cb.set_label("Permeability (mD)")

# Plot 4: Acoustic Impedance Well Data
sc = axes[1, 1].scatter(df['X'], df['Y'], c=df['AI'], cmap=cmap, edgecolors='black', alpha=0.8, s=marker_size)
axes[1, 1].set_title("Well Data - Acoustic Impedance", fontsize=14)
axes[1, 1].set_xlabel("X (m)", fontsize=12)
axes[1, 1].set_ylabel("Y (m)", fontsize=12)
axes[1, 1].set_aspect('equal', adjustable='box')  # Square plot
cb = plt.colorbar(sc, ax=axes[1, 1])
cb.set_label("Acoustic Impedance (kg/m2s * 10^6)")

# Adjust layout for clarity
plt.tight_layout()
plt.show()

#### Bivariate Analysis

Let's start with some simple bivariate plotting and calculations.

In [ ]:
plt.subplot(121)
plt.plot(df['Porosity'].values,df['Perm'].values, 'o', label='', markerfacecolor='red', markeredgecolor='black', alpha=0.2)
plt.title('Well Data Permeability vs. Porostiy')
plt.xlabel('Porosity (fraction)')
plt.ylabel('Permeability (mD)')
#plt.legend()

plt.subplot(122)
plt.plot(df['AI'].values,df['Porosity'].values, 'o', label='', markerfacecolor='red', markeredgecolor='black', alpha=0.2)
plt.title('Well Data Porostiy vs. Acoustic Impedance')
plt.ylabel('Porosity (fraction)')
plt.xlabel('Acoustic Impedance (m/s x g/cm^3)')

plt.subplots_adjust(left=0.0, bottom=0.0, right=2.2, top=1.2, wspace=0.2, hspace=0.2)
plt.show()

#### Correlation and Covariance

It is straight forward to calculate the covariance and correlation from the pairs of data in our dataset. Here's the covariance.  Notice that the matrix is symmetrical?  Makes sense, as the $C_{Por,Perm} = C_{Perm,Por}$.  Also, note that the diagonal values ($C_{i,j}$ where $i=j$) equal to the variance.  We check porosity by calculating the variance.

In [ ]:

# Compute and print the covariance matrix for columns 3 to 6
print(df.iloc[:, 3:7].cov())

# Compute the variance of 'Porosity'
porosity_variance = round(np.var(df['Porosity'].to_numpy()), 6)
print(f"The variance of porosity is {porosity_variance}")


Here are the correlation coefficients.

In [ ]:
df.iloc[:,3:7].corr()

#### Matrix Scatter Plots

If we have 3 or more variables to consider then matrix scatter plot offer an efficient method to display the multivariate relationships, 2 variables at a time.  We can identify:

1. the range, envelope of the paired data
2. homoscedastic and heteroscedastic behavoirs
3. non-linear features

Here's the seaborn package matrix scatter plot function, pairplot. Let's color the results by facies.

In [ ]:
sns.pairplot(df, hue='Facies',vars=['Facies','Porosity','Perm','AI'],markers='o')

### **Analysis of Seaborn Pairplot**

#### **1. Identifying the Range and Envelope of the Paired Data**
- **Range**: The axis limits of each scatter plot indicate the range of the variables:
  - **Porosity:** ~0.05 to ~0.2
  - **Perm (Permeability):** 0 to ~2000
  - **AI (Acoustic Impedance):** ~2000 to ~8000
- **Envelope (Data Spread and Bounds)**:
  - Scatter plots show the **spread of values** between two variables.
  - If data points are widely dispersed, the **envelope is broad**.
  - If data points are tightly clustered, the **envelope is narrow**.
  - **Density plots (KDEs) on the diagonal** provide insight into the distribution of each variable.

---

#### **2. Identifying Homoscedastic and Heteroscedastic Behavior**
- **Homoscedasticity (Constant Variance):**
  - If scatter plots show an **even spread of data points across all values of X**, then the data is **homoscedastic**.
  - Example: If **AI vs. Porosity** has a **consistent spread across all values**, it indicates **homoscedasticity**.

- **Heteroscedasticity (Changing Variance):**
  - If the **spread of data points increases or decreases** as the X variable changes, it suggests **heteroscedasticity**.
  - Example: 
    - **Perm vs. Porosity**: Shows an **increasing spread of permeability values as porosity increases**, indicating **heteroscedasticity**.
    - **AI vs. Porosity**: Displays a **funnel-like shape**, signaling a **variance shift**.

---

#### **3. Identifying Non-Linear Features**
- **Curved or Clustered Relationships in Scatter Plots:**
  - If data points **form a clear curve rather than a straight line**, this indicates a **non-linear relationship**.
  - Example:
    - **Perm vs. Porosity**: Shows a **non-linear trend**—as porosity increases, permeability increases in a **curved** rather than a straight-line fashion.
    - **AI vs. Porosity**: Also suggests a **non-linear effect**.

- **Divergence in KDE Distributions:**
  - The **density plots (diagonal histograms/KDEs)** reveal how different variables distribute across the Facies (0 vs. 1).
  - Example:
    - **Porosity distributions differ significantly for the two facies**, highlighting a **non-linear effect**.

---

#### **From the Pairplot:**
- **Heteroscedastic behavior** is visible in **Perm vs. Porosity** (increasing variance).
- **Non-linearity** is evident in **Perm vs. Porosity** and **AI vs. Porosity**.
- **The range/envelope** of each variable can be observed from the axis limits.


#### Joint, Conditional and Marginals

We can use kernel density estimation to estimate the joint probabilities density function (pdf) for the paired data, a 2D pdf! We could use this to estimate any required joint, marginal and conditional probability (care must be taken with normalization). Let's use the seaborn package 'kdeplot' function to estimate the joint pdf for porosity and acoustic impedance. 

In [ ]:

# Define colormap (modify as needed)
cmap = "plasma"

# Create KDE plot using the correct syntax
ax = sns.kdeplot(x=df['AI'].values, y=df['Porosity'].values, fill=True, levels=10, cmap=cmap)

# Add labels and title
ax.set_xlabel('Acoustic Impedance (m/s x g/cm^3)', fontsize=12)
ax.set_ylabel('Porosity (fraction)', fontsize=12)
ax.set_title('Porosity vs. Acoustic Impedance', fontsize=14)

# Show color bar
plt.colorbar(ax.collections[-1], label="Density")  # Add colorbar manually

# Show plot
plt.show()

The KDE-plot shows that most of the datapoints, the high-density region, are in the center. The plot also shows a negative correlation.

It is also useful to visualize the joint pdfs with the marginal pdfs on a single plot. We can use seaborn's 'jointgrid' to accomplish this. 

In [ ]:

# Define colormap
cmap = "plasma"

# Prepare the data: drop NaNs, reset index, and ensure numeric types
df = df.copy()
df = df[['AI', 'Porosity']].dropna().reset_index(drop=True)
df['AI'] = pd.to_numeric(df['AI'], errors='coerce')
df['Porosity'] = pd.to_numeric(df['Porosity'], errors='coerce')

# Convert columns to 1D NumPy arrays
x = df["AI"].values
y = df["Porosity"].values

# Create a JointGrid and plot the joint KDE contour (without the marginals)
g = sns.JointGrid(x=x, y=y, height=6)

# Plot the joint KDE contour
g.plot_joint(sns.kdeplot, levels=10, cmap=cmap, fill=False)

# Optionally, remove the marginal plots if you don't need them
g.ax_marg_x.set_visible(False)
g.ax_marg_y.set_visible(False)

# Adjust labels and title
g.ax_joint.set_xlabel("Acoustic Impedance (m/s × g/cm³)", fontsize=12)
g.ax_joint.set_ylabel("Porosity (fraction)", fontsize=12)
g.ax_joint.set_title("Porosity vs. Acoustic Impedance", fontsize=14)

plt.show()

The correlation coefficient and the p-value of the correlation coefficient (significant if < $\alpha/2$ or > $1-\alpha/2$). 

#### Calculating Conditional Statistics

Of course, we could just calculate the conditional statistics by-hand. We need to select some bins over the variable that we will condition to. Let's calculate conditional statistical of porosity given acoustic impedance. We will select 9 equal spaced bins.  

In [ ]:
AI_bins = np.linspace(2000,8000,10)            # set the bin boundaries and then the centroids for plotting
AI_centroids = np.linspace((AI_bins[0]+AI_bins[1])*0.5,(AI_bins[8]+AI_bins[9])*0.5,9)
print(AI_bins)                                 # check the boundaries
print(AI_centroids)                            # check the centroids
df['AI_bins'] = pd.cut(df['AI'], AI_bins,labels = AI_centroids) # cut on bondaries and lable with centroids 
df.head()                                      # check the new column in the DataFrame

Now we can use the 'groupby' function built-in to Pandas' DataFrames to extract subsets of porosity values in each bin from the DataFrame and then to calculate the conditional statistics: expectation, P90 and P10. Let's plot the result. 

In [ ]:
# Calculate the conditional expectations and quantiles
cond_exp = df.groupby('AI_bins')['Porosity'].mean()
cond_P90 = df.groupby('AI_bins')['Porosity'].quantile(0.9)
cond_P10 = df.groupby('AI_bins')['Porosity'].quantile(0.1)

# Convert the results to 1D NumPy arrays
cond_exp_arr = cond_exp.to_numpy()
cond_P90_arr = cond_P90.to_numpy()
cond_P10_arr = cond_P10.to_numpy()

# Plot the results (ensure AI_centroids is also a 1D array or list)
plt.figure()
plt.plot(AI_centroids, cond_exp_arr, color='black', label='Expectation')
plt.plot(AI_centroids, cond_P90_arr, 'r--', linewidth=1.0, label='P90')
plt.plot(AI_centroids, cond_P10_arr, 'r--', linewidth=1.0, label='P10')

plt.xlabel('Acoustic Impedance (m/s x g/cm^3)')
plt.ylabel('Porosity (fraction)')
plt.title('Porosity Conditional to Acoustic Impedance')
plt.ylim(pormin, pormax)
plt.xlim(AImin, AImax)

plt.text(3200, .10, 'P10')
plt.text(3200, .15, 'Expectation')
plt.text(3200, .19, 'P90')

plt.grid(True)
plt.legend()

plt.subplots_adjust(left=0.0, bottom=0.0, right=1.2, top=1.2, wspace=0.2, hspace=0.2)
plt.show()

Does acoustic impedance provide information about porosity? 

Yes, clearly the conditional statistics vary over acoustic impedance, knowing the acoustic impedance reduces  uncertainty about porosity.

#### Comments

This was a basic demonstration of multivariate analysis. A lot more could be done, for example, there are methods that reduce the dimensionality, and remove dependency to allow for independent variable modeling workflows etc.